In [1]:
from google.colab import drive
drive.mount('/content/drive')
BATCH_SIZE = 128
CROP_SIZE = 60
TRAIN_DIR = "/content/drive/MyDrive/dataset/train/"
TEST_DIR = "/content/drive/MyDrive/dataset/test/"
OUTDIR = "model"

Mounted at /content/drive


In [2]:
pip install jcopdl

  Created wheel for jcopdl: filename=jcopdl-1.1.9-py2.py3-none-any.whl size=16925 sha256=7097ea0d0232a5bb23fe983e5cdff224c1df95e8475ab43c56842106b5c05ca4
  Stored in directory: /root/.cache/pip/wheels/bb/fd/9d/fc72bcf569000eabd487e654fdf55b587e66882b60eaa5693b
Successfully built jcopdl


In [3]:
from torch import nn
from jcopdl.layers import conv_block, linear_block

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            # conv_block(3, 8),
            # conv_block(8, 16),
            conv_block(3, 32),
            conv_block(32, 64),            
            nn.Flatten()
        )
        
        self.fc = nn.Sequential(
            linear_block(64*15*15, 960, dropout=0.1),
            linear_block(960, 3, activation="lsoftmax")
        )
        
    def forward(self, x):
        return self.fc(self.conv(x))

In [4]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [5]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# def train():
    # Dataset & Dataloader
train_transform = transforms.Compose([
    # transforms.RandomRotation(15),
    transforms.RandomResizedCrop(CROP_SIZE, scale=(0.8, 1.0)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(76),
    transforms.CenterCrop(CROP_SIZE),
    transforms.ToTensor()
])

train_set = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
trainloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

test_set = datasets.ImageFolder(TEST_DIR, transform=test_transform)
testloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

label2cat = train_set.classes
label2cat

# Config
config = set_config({
    "batch_size": BATCH_SIZE,
    "crop_size": CROP_SIZE
})

# Training Preparation
model = CNN().to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
callback = Callback(model, config, outdir=OUTDIR)

# Training
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn("test", test_set, testloader, model, criterion, optimizer, device)

    # Callbacks
    callback.log(train_cost, test_cost, train_score, test_score)
    callback.save_checkpoint()
    if callback.early_stopping(model, monitor="test_score"):
        callback.plot_cost()
        callback.plot_score()
        break
            
# if __name__ == "__main__":
#     train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored

In [ ]:
label2cat

In [ ]:
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)

In [ ]:
with torch.no_grad():
    model.eval()
    output = model(feature)
    preds = output.argmax(1)
preds

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(4,4,figsize=(24,24))
for image, label, pred, ax in zip(feature, target, preds, axes.flatten()):
  ax.imshow(image.permute(1,2,0).cpu())
  font = {'color':'r'} if label != pred else {'color':'g'}
  label, pred = label2cat[label.item()], label2cat[pred.item()]
  ax.set_title(f"L: {label} | P: {pred}", fontdict=font)
  ax.axis("off");

In [ ]:
# f, z = next(iter(testloader))
# f, z = f.to(device), z.to(device)
# with torch.no_grad():
#     model.eval()
#     output = model(f)
#     preds = output.argmax(1)
# print(preds)
# plt.imshow(f[1][1])
# print(f[3][1])
# # plt.imshow(f[2][3])
# fig, axes = plt.subplots(4,4,figsize=(24,24))
# for im, label, ax in zip(f, z, axes.flatten()):
#   # print(im.shape)
#   ax.imshow(im.permute(1,2,0).cpu())
#   ax.axis("off");